In [99]:
def cost_data_to_hierarchy(cost_data):
    # Build a nested dict structure from cost_data
    def insert_path(tree, path, value):
        node = tree
        for key in path[:-1]:
            node = node.setdefault(key, {"children": {}, "sum": 0})
            node["sum"] += value
            node = node["children"]
        leaf = path[-1]
        node.setdefault(leaf, {"children": {}, "sum": 0})
        node[leaf]["sum"] += value

    # Build the tree and compute total sum
    tree = {}
    total = 0
    for d in cost_data:
        for k, v in d.items():
            insert_path(tree, k, v)
            total += v

    # Recursively compute percentages and clean up
    def compute_percentages(node, parent_sum, top_sum, is_top_level=False):
        node_sum = node.get("sum", 0)
        # category_percentage: for top-level, use total; for others, use top_sum
        if is_top_level:
            node["category_percentage"] = round(100 * node_sum / total, 1) if total else 100.0
        else:
            node["category_percentage"] = round(100 * node_sum / top_sum, 1) if top_sum else 100.0
        node["total_percentage"] = round(100 * node_sum / total, 1) if total else 100.0
        children = node.get("children", {})
        for key, child in children.items():
            compute_percentages(child, node_sum, top_sum, False)
        node.pop("sum", None)
        if children:
            node["children"] = dict(children)
        else:
            node.pop("children", None)

    # Compute percentages for all top-level nodes
    for key, node in tree.items():
        compute_percentages(node, total, node.get("sum", 0), is_top_level=True)

    return tree


def plot_cost_treemap(cost_data, title):
    total_cost = sum(list(d.values())[0] for d in cost_data)
    title = f"{title} - Total Costs: ${total_cost:,.0f}"    
    cost_hierarchy = cost_data_to_hierarchy(cost_data)
    import pandas as pd
    import plotly.express as px

    def build_data_from_hierarchy(hierarchy, cat1_label=None, cat2_label=None):
        data = []
        for key, value in hierarchy.items():
            # If this node has children, recurse
            if "children" in value:
                label = key
                if "category_percentage" in value:
                    label += f"({value['category_percentage']}%)"
                if cat1_label is None:
                    # Top level: Global Costs or Per DB Costs
                    data += build_data_from_hierarchy(value["children"], label, None)
                else:
                    # Second level: Fixed Costs or Variable Costs
                    data += build_data_from_hierarchy(value["children"], cat1_label, key)
            else:
                # Leaf node
                cat3_label = f"{key.replace(' & ', '<br>& ').replace(' ', '<br>', 1)}<br>({value['category_percentage']}%)"
                data.append((cat1_label, cat2_label if cat2_label else "", cat3_label, value["total_percentage"]))
        return data

    data = build_data_from_hierarchy(cost_hierarchy)
    df = pd.DataFrame(data, columns=["Cat 1", "Cat 2", "Cat 3", "Percentage"])
    df["Label"] = df["Cat 3"]
    from pprint import pprint
    pprint(df)

    fig = px.treemap(
        df,
        path=["Cat 1", "Cat 2", "Label"],
        values="Percentage",
        title=title
    )
    fig.update_layout(height=600)
    fig.update_layout(width=1300)
    fig.show()

In [93]:

cost_data = [
    {("Global Costs", "Monitoring & Alerting"): 11285.00},
    {("Global Costs", "Auth"): 5997.50},
    {("Global Costs", "Billing"): 3594.00},
    {("Global Costs", "Payment"): 5997.50},
    {("Global Costs", "prod-saas"): 18479.44},
    {("Global Costs", "Security, intrusion detection, etc"): 21266.29},
    {("Global Costs", "VPC"): 13409.11},
    {("Per DB Costs", "Fixed Costs", "EBS"): 42155.73},
    {("Per DB Costs", "Fixed Costs", "Access Node"): 16262.51},
    {("Per DB Costs", "Variable Costs", "EC2"): 219295.62},
    {("Per DB Costs", "Variable Costs", "DB S3 Storage"): 38972.10},
    {("Per DB Costs", "Variable Costs", "Traffic"): 5280.85},
    {("Per DB Costs", "Variable Costs", "DB S3 Storage API Requests"): 31056.92},
    {("Per DB Costs", "Variable Costs", "Offline Backups S3 Storage"): 16738.38},
    {("Per DB Costs", "Variable Costs", "Offline Backups S3 API Requests"): 4108.13},
    {("Per DB Costs", "Variable Costs", "VPC"): 17852.11},
    {("Per DB Costs", "Variable Costs", "Other"): 1084.30}
]


plot_cost_treemap(cost_data, "SaaS Cost Breakdown Jan-May - Credits Cosumed 415,906 (excl. Wooga)")

In [101]:
cost_data = [
    {("Global Costs", "Monitoring & Alerting"): 4736.83},
    {("Global Costs", "Auth"): 1198.00},
    {("Global Costs", "Billing"): 5916.67},
    {("Global Costs", "Payment"): 1198.00},
    {("Global Costs", "prod-saas"): 6379.77},
    {("Global Costs", "Security"): 5217.81},
    {("Global Costs", "VPC"): 5214.60},
    {("Global Costs", "Other"): 1247.66},
    {("Per DB Costs", "Fixed Costs", "EBS"): 15672.85},
    {("Per DB Costs", "Fixed Costs", "Access Node"): 6858.97},
    {("Per DB Costs", "Variable Costs", "EC2"): 14900.85},
    {("Per DB Costs", "Variable Costs", "DB S3 Storage"): 5032.65},
    {("Per DB Costs", "Variable Costs", "Traffic"): 1258.20},
    {("Per DB Costs", "Variable Costs", "DB S3 Storage API Requests"): 9302.15},
    {("Per DB Costs", "Variable Costs", "Offline Backup S3 Storage"): 4738.46},
    {("Per DB Costs", "Variable Costs", "Offline Backup S3 API Request"): 1622.06},
    {("Per DB Costs", "Variable Costs", "VPC"): 7036.67},
    {("Per DB Costs", "Variable Costs", "Other"): 576.95}
]


plot_cost_treemap(cost_data, "SaaS Cost Breakdown May-June - Credits Cosumed (TBC) 167,116")

                  Cat 1           Cat 2  \
0   Global Costs(31.7%)                   
1   Global Costs(31.7%)                   
2   Global Costs(31.7%)                   
3   Global Costs(31.7%)                   
4   Global Costs(31.7%)                   
5   Global Costs(31.7%)                   
6   Global Costs(31.7%)                   
7   Global Costs(31.7%)                   
8   Per DB Costs(68.3%)     Fixed Costs   
9   Per DB Costs(68.3%)     Fixed Costs   
10  Per DB Costs(68.3%)  Variable Costs   
11  Per DB Costs(68.3%)  Variable Costs   
12  Per DB Costs(68.3%)  Variable Costs   
13  Per DB Costs(68.3%)  Variable Costs   
14  Per DB Costs(68.3%)  Variable Costs   
15  Per DB Costs(68.3%)  Variable Costs   
16  Per DB Costs(68.3%)  Variable Costs   
17  Per DB Costs(68.3%)  Variable Costs   

                                         Cat 3  Percentage  \
0       Monitoring<br>&<br>Alerting<br>(15.2%)         4.8   
1                               Auth<br>(3.9%)         1.2

In [100]:
cost_data = [
    {("Fixed Costs", "EBS"): 383.64},
    {("Fixed Costs", "Access Node"): 246.84},
    {("Variable Costs", "EC2"): 6581.98},
    {("Variable Costs", "DB S3 Storage"): 824.00},
    {("Variable Costs", "Traffic"): 22.45},
    {("Variable Costs", "DB S3 Storage API Requests"): 1037.31},
    {("Variable Costs", "Offline Backup S3 Storage"): 2001.55},
    {("Variable Costs", "Offline Backup S3 API Request"): 429.54},
    {("Variable Costs", "VPC"): 152.34},
    {("Variable Costs", "Other"): 21.52},
]


plot_cost_treemap(cost_data, "Adena - May-June - Credits Consumed: 37,100")

                   Cat 1 Cat 2                                       Cat 3  \
0      Fixed Costs(5.4%)                                    EBS<br>(60.8%)   
1      Fixed Costs(5.4%)                         Access<br>Node<br>(39.2%)   
2  Variable Costs(94.6%)                                    EC2<br>(59.5%)   
3  Variable Costs(94.6%)                        DB<br>S3 Storage<br>(7.4%)   
4  Variable Costs(94.6%)                                 Traffic<br>(0.2%)   
5  Variable Costs(94.6%)           DB<br>S3 Storage API Requests<br>(9.4%)   
6  Variable Costs(94.6%)           Offline<br>Backup S3 Storage<br>(18.1%)   
7  Variable Costs(94.6%)        Offline<br>Backup S3 API Request<br>(3.9%)   
8  Variable Costs(94.6%)                                     VPC<br>(1.4%)   
9  Variable Costs(94.6%)                                   Other<br>(0.2%)   

   Percentage                                       Label  
0         3.3                              EBS<br>(60.8%)  
1         2.1        

In [89]:
cost_data = [
    {("Fixed Per DB Costs", "EBS"): 1125.55},
    {("Fixed Per DB Costs", "Access Node"): 977.49},
    {("Variable Per DB Costs", "EC2"): 5424.28},
    {("Variable Per DB Costs", "DB S3 Storage"): 475.38},
    {("Variable Per DB Costs", "Traffic"): 334.85},
    {("Variable Per DB Costs", "DB S3 Storage API Requests"): 744.84},
    {("Variable Per DB Costs", "Offline Backup S3 Storage"): 598.11},
    {("Variable Per DB Costs", "Offline Backup S3 API Request"): 518.15},
    {("Variable Per DB Costs", "VPC"): 436.04},
    {("Variable Per DB Costs", "Other"): 39.45},    
]

plot_cost_treemap(cost_data, f"Formue May-June - Credits Consumed: 26,419")

In [96]:
cost_data = [
    {("Fixed Per DB Costs", "EBS"): 0},
    {("Fixed Per DB Costs", "Access Node"): 182.94},
    {("Variable Per DB Costs", "EC2"): 1.70},
    {("Variable Per DB Costs", "DB S3 Storage"): 0.01},
    {("Variable Per DB Costs", "Traffic"): 0.28},
    {("Variable Per DB Costs", "DB S3 Storage API Requests"): 20},
    {("Variable Per DB Costs", "Offline Backup S3 Storage"): 0},
    {("Variable Per DB Costs", "Offline Backup S3 API Request"): 0.03},
    {("Variable Per DB Costs", "VPC"): 84.43},
    {("Variable Per DB Costs", "Other"): 1.96},
]

plot_cost_treemap(cost_data, "Trial Inet AB acc_DknhzhGiRJ2DTC3fizCTCw (not using DB)")

In [ ]:
import plotly.express as px
import pandas as pd
from pprint import pprint

df = pd.DataFrame({
    'cat1': ['A', 'A', 'B', 'B'],
    'cat2': ['X', 'Y', 'X', 'Y'],
    'percentage': [20, 30, 25, 25],
    'label': ['AX', 'AY', 'BX', 'BY']
})
pprint(df)

fig = px.treemap(
    df,
    path=['cat1', 'cat2', 'label'],    # Hierarchy: cat1 > cat2 > label
    values='percentage',               # Size of boxes
)

fig.show()